In [116]:
#pip install mysql-connector-python
import mysql.connector
import time

In [118]:

# Database connection parameters
db_config = {
    "host": "localhost",
    "user": "greg",
    "password": "greg",
    "database": "online_store"
}

# Connect to the MySQL server
db_conn = mysql.connector.connect(**db_config)
cursor = db_conn.cursor()

In [6]:
# Function to create/add a new product to DB
def add_product(product_data):
    try:
        insert_query = "INSERT INTO Products (ProductName, Description, Price, Category, StockQuantity, Manufacturer, ReleaseDate, ImageURL) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(insert_query, product_data)
        db_conn.commit()
        return cursor.lastrowid  # Return the ID of the newly created product
    except Exception as e:
        print("Error creating product:", e)
        db_conn.rollback()


# Example usage
product_data = ("Amplifier", "Precision Power Amplifier.", 99.99, "Electronics", 100, "Diora", "2023-10-11", "sample_image.jpg")
new_product_id = add_product(product_data)
print("New product ID:", new_product_id)


New product ID: 86


In [ ]:
# Function to read product information by ID or select a random product
def read_product(product_id=None):
    try:
        if product_id is None:
            select_query = "SELECT * FROM Products ORDER BY RAND() LIMIT 1"
            cursor.execute(select_query)
        else:
            select_query = "SELECT * FROM Products WHERE ProductID = %s"
            cursor.execute(select_query, (product_id,))

        # Get the column names from the cursor description
        column_names = [desc[0] for desc in cursor.description]

        product_data = cursor.fetchone()
        if product_data:
            # Convert the result to a dictionary with column names as keys
            product_dict = dict(zip(column_names, product_data))
            return product_dict
        else:
            return None

    except Exception as e:
        print("Error reading product information:", e)

# Example usage for reading a product by ID or selecting a random product
# Reading a product by ID
product_id = 2  # example ProductID
product_info = read_product(product_id)
if product_info:
    print("Product Information:", product_info)

# Selecting a random product
random_product_info = read_product()
if random_product_info:
    print("Random Product Information:", random_product_info)

In [13]:
# Function to update product price
def update_product_price(product_id, updated_data):
    try:
        update_query = "UPDATE Products SET Price = %s WHERE ProductID = %s"
        cursor.execute(update_query, (updated_data, product_id))
        db_conn.commit()
    except Exception as e:
        print("Error updating product:", e)
        db_conn.rollback()

# Example usage
product_info= update_product_price(1, 79.99)
print("Random Product Info:", product_info)

Random Product Info: None


In [ ]:
# Function to update product information with multiple parameters
def update_product(product_id, **kwargs):
    """ 
    allowed kwargs keys:
    - ProductName VARCHAR(255), 
    - Description TEXT,
    - Price DECIMAL(10, 2)
    - Category VARCHAR(50) 
    - StockQuantity INT 
    - Manufacturer VARCHAR(100),
    - ReleaseDate DATE,
    - ImageURL VARCHAR(255)
    """
    try:
        # Construct the SQL query and a list of values to update
        update_query = "UPDATE Products SET "
        update_values = []

        for key, value in kwargs.items():
            update_query += f"{key} = %s, "
            update_values.append(value)

        update_query = update_query.rstrip(", ")  # Remove the trailing comma and space
        update_query += " WHERE ProductID = %s"
        update_values.append(product_id)

        cursor.execute(update_query, update_values)
        db_conn.commit()

        # Retrieve the updated product data
        updated_data = read_product(product_id)
        return updated_data 

    except Exception as e:
        print("Error updating product:", e)
        db_conn.rollback()

# Example usage to update multiple parameters
update_product(2, Price=79.99, StockQuantity=50, Description="Updated description")


In [ ]:
# Function to delete a product
# if DB nas not set ON DELETE CASCADE or ON DELETE SET NULL then error
def delete_product(product_id):
    try:
        # Fetch the product data before deleting it
        select_query = "SELECT * FROM Products WHERE ProductID = %s"
        cursor.execute(select_query, (product_id,))
        deleted_data = cursor.fetchone()

        # Delete the product
        delete_query = "DELETE FROM Products WHERE ProductID = %s"
        cursor.execute(delete_query, (product_id,))
        db_conn.commit()

        return deleted_data
    
    except mysql.connector.errors.IntegrityError as e:
        # Handle foreign key constraint error
        print("Error deleting product due to foreign key constraint:", e)
        db_conn.rollback()
        return None

    except Exception as e:
        print("Error deleting product:", e)
        db_conn.rollback()

# Example usage
delete_product(2)

In [ ]:
# Function to list all products
def list_all_products(list_categories=False, category_name=None):
    # list avalible categories
    if list_categories:
        # Fetch and return the list of unique categories
        try:
            cursor.execute("SELECT DISTINCT Category FROM Products")
            categories = cursor.fetchall()
            categories = [category[0] for category in categories]
            return categories
        
        except Exception as e:
            print("Error fetching category list:", e)
            return []

    elif category_name:
        # Fetch and return the list of products in the specified category
        try:
            cursor.execute("SELECT ProductID, ProductName, Price FROM Products WHERE Category = %s", (category_name,))
            products = cursor.fetchall()
            return products
        
        except Exception as e:
            print(f"Error fetching products in the '{category_name}' category:", e)
            return []

    else:
        # Fetch and return the list of all products
        try:
            cursor.execute("SELECT ProductID, ProductName, Price FROM Products")
            products = cursor.fetchall()
            return products
        except Exception as e:
            print("Error fetching all products:", e)
            return []

# Example usages
# List categories
categories = list_all_products(list_categories=True)
print("Categories:", categories)

# List products in a specific category
category_name = "Electronics"  # Change this to the desired category
products = list_all_products(category_name=category_name)
print(f"Products in the '{category_name}' category:", products)

# List all products
all_products = list_all_products()
print("All Products:", all_products)

In [ ]:
# Function to get product details by product ID and return a product (dictionary)
def get_product_details(product_id):
    try:
        select_query = "SELECT * FROM Products WHERE ProductID = %s"
        cursor.execute(select_query, (product_id,))
        product_data = cursor.fetchone()

        if product_data:
            # Get the column names from the cursor description
            column_names = [desc[0] for desc in cursor.description]

            # Create a dictionary using column names as keys
            product_dict = dict(zip(column_names, product_data))
            return product_dict
        else:
            return None
    except Exception as e:
        print("Error fetching product details:", e)

# Example usage to print the dictionary
product_id = 1  # Change this to the another ProductID
product_details = get_product_details(product_id)

if product_details:
    print("Product Details:")
    for key, value in product_details.items():
        print(f"{key} : {value}")
else:
    print("Product not found.")

In [ ]:
# Function to display product reviews
def display_product_reviews(product_id):
    select_query = "SELECT Rating, Comment, ReviewDate FROM Reviews WHERE ProductID = %s"
    cursor.execute(select_query, (product_id,))
    reviews = cursor.fetchall()
    for review in reviews:
        rating, comment, review_date = review
        print(f"Rating: {rating}, Review Date: {review_date}, Comment: {comment}")

# Example usage for displaying product reviews
display_product_reviews(19)

In [ ]:
# Function to display product review and avg, sort reviews if necessary (asc/desc)
def display_product_reviews(product_id, sorting_direction=None):
    try:
        # base query to fetch reviews, with no sorting
        select_query = "SELECT Rating, Comment, ReviewDate FROM Reviews WHERE ProductID = %s"
        # apply sorting if a direction is provided
        # Define a mapping for sorting direction
        sorting_direction_map = {  "asc": "ASC", "desc": "DESC" }
        # add sorting direction
        if sorting_direction and sorting_direction in sorting_direction_map:
            select_query += f" ORDER BY Rating { sorting_direction_map[sorting_direction] }"

        cursor.execute(select_query, (product_id,))
        reviews = cursor.fetchall()
        total_ratings = 0
        num_reviews = len(reviews)

        if num_reviews > 0:
            for review in reviews:
                rating, comment, review_date = review
                total_ratings += rating
                print(f"Rating: {rating}, Review Date: {review_date}, Comment: {comment}")

            average_rating = total_ratings / num_reviews
            print(f"\nAverage Rating: {average_rating:.2f}")
        else:
            print("No reviews found for this product.")
    except Exception as e:
        print("Error fetching and displaying product reviews:", e)

# Example usage for displaying sorted product reviews and calculating average score
product_id = 53  # Change this to the desired ProductID
#display_product_reviews(product_id)
sorting_direction = "asc4"  # Change this to "asc" or "desc" for sorting, or set to None for no sorting
display_product_reviews(product_id, sorting_direction)
#display_product_reviews(product_id)


In [ ]:
import bcrypt  # Import the bcrypt library

# Function to create a new consumer with a hashed password
def create_consumer(username, password, first_name, last_name, email, phone, address):
    # Generate a password hash using bcrypt
    password_hash = bcrypt.hashpw(password.encode("utf-8"), bcrypt.gensalt())

    try:
        insert_query = "INSERT INTO Customers (Username, PasswordHash, FirstName, LastName, Email, Phone, Address, RegistrationDate) VALUES (%s, %s, %s, %s, %s, %s, %s, NOW())"
        cursor.execute(insert_query, (username, password_hash, first_name, last_name, email, phone, address))
        db_conn.commit()
        return cursor.lastrowid  # Return the ID of the newly created customer
    except Exception as e:
        print("Error creating customer:", e)
        db_conn.rollback()

# Example usage for creating a new consumer
new_consumer_id = create_consumer("new_user", "password123", "John", "Doe", "john@example.com", "123-456-7890", "123 Main St")
if new_consumer_id:
    print(f"New consumer created with ID {new_consumer_id}")

In [ ]:
# Function to get consumer data
def get_customer_data(customer_id):
    try:
        # Fetch customer information from the database
        select_query = "SELECT * FROM Customers WHERE CustomerID = %s"
        cursor.execute(select_query, (customer_id,))
        customer_data = cursor.fetchone()

        if customer_data:
            # Define the column names to use as keys in the dictionary
            column_names = [desc[0] for desc in cursor.description]

            # Create a dictionary with column names as keys and customer data as values
            customer_dict = dict(zip(column_names, customer_data))
            return customer_dict
        else:
            return None
    except Exception as e:
        print("Error fetching customer data:", e)

# Example usage for getting customer data
customer_id = 2  # Change this CustomerID
customer_info = get_customer_data(customer_id)

if customer_info:
    print("Customer Data:", customer_info)
else:
    print("Customer not found.")

In [64]:
# Function to count max of records in tables
def count_entities(entity_type):
    if entity_type == "customers":
        query = "SELECT COUNT(*) FROM Customers"
    elif entity_type == "orders":
        query = "SELECT COUNT(*) FROM Orders"
    elif entity_type == "products":
        query = "SELECT COUNT(*) FROM Products"
    else:
        print("Invalid entity type. Use 'customers', 'orders', or 'products'.")
        return None

    try:
        cursor.execute(query)
        count = cursor.fetchone()[0]
        return count
    except Exception as e:
        print(f"Error counting {entity_type}: {e}")
        return None

# Example usage to count customers, orders, or products
entity_type = "customers"  # Change this to 'orders' or 'products' to count those entities
count = count_entities(entity_type)

if count is not None:
    print(f"Number of {entity_type.capitalize()}: {count}")

Number of Products: 92


In [ ]:
# Function to create a new single item order
def create_order(customer_id, product_id, quantity):

    # Fetch the product price from the database
    product_info = read_product(product_id)
    
    if product_info:
        product_price = product_info["Price"] 
        total_amount = product_price * quantity

        try:
            # Insert a new order in the Orders table, with total amount calculated
            insert_query = "INSERT INTO Orders (CustomerID, OrderDate, TotalAmount, OrderStatus) VALUES (%s, NOW(), %s, 'Pending')"
            cursor.execute(insert_query, (customer_id, total_amount))
            order_id = cursor.lastrowid  # Get the ID of the newly created order

            # Create an order item
            insert_order_item_query = "INSERT INTO OrderItems (OrderID, ProductID, Quantity, ItemPrice) VALUES (%s, %s, %s, %s)"
            cursor.execute(insert_order_item_query, (order_id, product_id, quantity, total_amount))
            db_conn.commit()
            return order_id
        
        except Exception as e:
            print("Error creating order:", e)
            db_conn.rollback()
    else:
        print("Product not found.")
        return None

# Example usage for creating an order
order_id = create_order(1, 23, 2)  # CustomerID 1 orders 2 units of ProductID 1
if order_id:
    print(f"New order created with ID {order_id}")
else:
    print("Something went wrong. Try again")

In [ ]:
# Function to create a new multiple-item order
def create_order(customer_id, order_items):
    try:
        # Begin a transaction
        # db_conn.start_transaction()

        # Check if a transaction is already active
        if not db_conn.in_transaction:
            # Begin a new transaction
            db_conn.start_transaction()

        # Initialize the total amount
        total_amount = 0

        # Insert a new order in the Orders table, with total amount = 0
        insert_order_query = "INSERT INTO Orders (CustomerID, OrderDate, TotalAmount, OrderStatus) VALUES (%s, NOW(), 0, 'pending')"
        cursor.execute(insert_order_query, (customer_id,))
        order_id = cursor.lastrowid  # Get the order ID

        # Process each item in the order
        for item in order_items:
            product_id, quantity = item

            # Fetch the product price from the database
            select_product_price_query = "SELECT Price FROM Products WHERE ProductID = %s"
            cursor.execute(select_product_price_query, (product_id,))
            product_price = cursor.fetchone()[0]

            # Calculate the item price
            item_total_cost = product_price * quantity

            # Insert order item
            insert_order_item_query = "INSERT INTO OrderItems (OrderID, ProductID, Quantity, ItemPrice) VALUES (%s, %s, %s, %s)"
            cursor.execute(insert_order_item_query, (order_id, product_id, quantity, item_total_cost))

            # Add the item's cost to the total amount
            total_amount += item_total_cost
            # Wait for 1 second
            time.sleep(0.5)

        # Update the total amount in the order
        update_order_query = "UPDATE Orders SET TotalAmount = %s WHERE OrderID = %s"
        cursor.execute(update_order_query, (total_amount, order_id))

         # Commit the transaction if it was started in this function
        if not db_conn.in_transaction:
            db_conn.commit()
        
        # Commit the transaction
        #db_conn.commit()

        return order_id
    except Exception as e:
        print("Error creating order:", e)
        db_conn.rollback()

# Example usage for creating an order
customer_id = 38
order_items = [(11, 3), (35, 1)]  # ProductID and quantity
new_order_id = create_order(customer_id, order_items)
if new_order_id:
    print("New Order ID:", new_order_id)

In [ ]:
# Function to get all order details
def get_order_details(order_id):
    try:
        # Query to retrieve order details, customer information, and product information
        select_query = """
        SELECT Orders.OrderID, Orders.OrderDate, Orders.TotalAmount, Orders.OrderStatus,
               Customers.CustomerID, Customers.FirstName, Customers.LastName, Customers.Email
        FROM Orders
        JOIN Customers ON Orders.CustomerID = Customers.CustomerID
        WHERE Orders.OrderID = %s
        """
        
        cursor.execute(select_query, (order_id,))
        order_data = cursor.fetchall()
        
        if order_data:
            # Get the column names from the cursor description
            column_names = [desc[0] for desc in cursor.description]
            order_details = dict(zip(column_names, order_data[0]))
            order_details['OrderItems'] = []  # Add an empty list for order items

            # Query to retrieve order items with quantity and price
            order_items_query = """
            SELECT OrderItems.ProductID, Products.ProductName, OrderItems.Quantity, OrderItems.ItemPrice
            FROM OrderItems
            JOIN Products ON OrderItems.ProductID = Products.ProductID
            WHERE OrderItems.OrderID = %s
            """
            cursor.execute(order_items_query, (order_id,))
            order_items = cursor.fetchall()
            for item in order_items:
                item_dict = dict(zip(['ProductID', 'ProductName', 'Quantity', 'ItemPrice'], item))
                order_details['OrderItems'].append(item_dict)

            return order_details
        else:
            return None
    except Exception as e:
        print("Error fetching order details:", e)

# Example usage to get and print the order details
order_id = 211  # Change this to the desired OrderID
order_details = get_order_details(order_id)

if order_details:
    print("Order Details:")
    for key, value in order_details.items():
        if key == 'OrderItems':
            print("Order Items:")
            for item in value:
                items_ordered =""
                for item_key, item_value in item.items():
                    items_ordered += f"{item_key} : {item_value}, "
                    # print(f"{item_key} : {item_value}")
                print(f"{items_ordered}")     # print indyvidual items order.
            continue
        print(f"{key} : {value}")           # print order details
        
    total_amount = order_details['TotalAmount']
    print(f"Total Price: {total_amount}")
else:
    print("Order not found.")


In [ ]:
# Function to retrieve customer orders
def get_customer_orders(customer_id):
    select_query = "SELECT * FROM Orders WHERE CustomerID = %s"
    cursor.execute(select_query, (customer_id,))
    return cursor.fetchall()

# Example usage
customer_id = 46  # Change CustomerID
customer_orders = get_customer_orders(customer_id=customer_id)
if isinstance(customer_orders, list):
    print("Customer Orders:")
    for order in customer_orders:
        print(order)
elif isinstance(customer_orders, str):
    print(customer_orders)

In [ ]:
# Function to check order status
def check_order_status(order_id):
    select_query = "SELECT OrderStatus FROM Orders WHERE OrderID = %s"
    cursor.execute(select_query, (order_id,))
    order_status = cursor.fetchone()
    if order_status:
        return order_status[0]
    else:
        return "Order not found"
    
# Example usage for checking and changing order status
order_id = 4  # Change this to an existing order ID

# Check order status
status = check_order_status(order_id)
print(f"Order Status: {status}")

In [ ]:
# Function to change order status
def change_order_status(order_id, new_status):
    try:
        update_query = "UPDATE Orders SET OrderStatus = %s WHERE OrderID = %s"
        cursor.execute(update_query, (new_status, order_id))
        db_conn.commit()
        return True
    except Exception as e:
        print("Error changing order status:", e)
        db_conn.rollback()
        return False

# Change order status
order_id = 2
if change_order_status(order_id, "Shipped"):
    print("Order status changed successfully.")
else:
    print("Failed to change order status.")

In [ ]:
# Function to perform either a check of the order status or a change in order status based on the argument provided
order_statuses = ["Pending", "Paid", "Shipped", "Delivered"]

def order_status(order_id, new_status=None):
    try:
        if new_status is not None:
            # Change order status
            update_query = "UPDATE Orders SET OrderStatus = %s WHERE OrderID = %s"
            cursor.execute(update_query, (new_status, order_id))
            db_conn.commit()
            return True
        else:
            # Check order status
            order_id=[order_id]
            select_query = "SELECT OrderStatus FROM Orders WHERE OrderID = %s"
            cursor.execute(select_query, (order_id))
            order_status = cursor.fetchone()
            if order_status:
                return order_status[0]
            else:
                return "Order not found"
    except Exception as e:
        print("Error checking/changing order status:", e)
        db_conn.rollback()
        return False

# Example usage
# Check order status
order_id = 4
current_status = order_status(order_id)
if isinstance(current_status, str):
    print(current_status)
else:
    print("Current order status:", current_status)

# Change order status
order_id = 4
new_status = "Shipped"
if order_status(order_id, new_status):
    print(f"Order Id: {order_id} status changed successfully to {new_status}.")
else:
    print("Failed to change order status.")


In [110]:
import random

from faker import Faker
import bcrypt  # Import the bcrypt librar
# Initialize Faker 
fake = Faker()

# Function to make a payment for a specific order
def make_payment(order_id, payment_amount, payment_method, transaction_id):
    try:
        # Check if the order exists
        select_order_query = "SELECT * FROM Orders WHERE OrderID = %s"
        cursor.execute(select_order_query, (order_id,))
        order = cursor.fetchone()
        if not order:
            print("Order not found.")
            return False

        # Insert the payment information
        insert_payment_query = "INSERT INTO Payments (OrderID, PaymentDate, PaymentAmount, PaymentMethod, TransactionID) VALUES (%s, NOW(), %s, %s, %s)"
        cursor.execute(insert_payment_query, (order_id, payment_amount, payment_method, transaction_id))
        db_conn.commit()

        # Update the order status to "Paid"
        update_order_status_query = "UPDATE Orders SET OrderStatus = 'Delivered' WHERE OrderID = %s"
        cursor.execute(update_order_status_query, (order_id,))
        db_conn.commit()

        return True
    except Exception as e:
        print("Error making payment:", e)
        db_conn.rollback()
        return False

# Example usage for making a payment
payment_methods =["Card", "PayPal", "Wire"]
# Option to Make a Payment for given Order ID.
order_id = 29
order_details = get_order_details(order_id)

if order_details:
    print("Order Details:")
    print(order_details)
    payment_amount = order_details["TotalAmount"]
    payment_methods =["Card", "PayPal", "Wire"]
    payment_method = random.choice(payment_methods)
    order_id_status = order_details["OrderStatus"]
    transaction_id = fake.uuid4()
    # Create Payment, if sucesfull. change order status to Paid.
    if make_payment(order_id, payment_amount, payment_method, transaction_id):
        print("Payment made successfully.")
        # Change Order Status to Paid
        if change_order_status(order_id, "Paid"): 
            print("Order status changed to 'Paid'.")
        else:
            print("Failed to change order status to 'Paid'.")
    else:
        print("Payment failed.")

else:
    print("Order not found.")


In [ ]:
import pandas as pd

def denormalize_data_to_dataframe():
    try:
        # Execute SQL queries to fetch data from multiple tables and denormalize it
        query = """
        SELECT Orders.OrderID, Orders.OrderDate, Orders.TotalAmount, Orders.OrderStatus, 
               Customers.CustomerID, Customers.FirstName, Customers.LastName, Customers.Email, 
               OrderItems.ProductID, Products.ProductName, OrderItems.Quantity, OrderItems.ItemPrice
        FROM Orders
        JOIN Customers ON Orders.CustomerID = Customers.CustomerID
        JOIN OrderItems ON Orders.OrderID = OrderItems.OrderID
        JOIN Products ON OrderItems.ProductID = Products.ProductID
        """
        cursor.execute(query)

        # Fetch the result and column names
        results = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

        # Create a Pandas DataFrame with the denormalized data
        df = pd.DataFrame(results, columns=column_names)

        return df

    except Exception as e:
        print("Error fetching and denormalizing data:", e)

    finally:
        # Close the cursor and database connection
        cursor.close()
        db_conn.close()

# Call the function to denormalize and retrieve the data in a Pandas DataFrame
data_df = denormalize_data_to_dataframe()

# Display the first few rows of the Pandas DataFrame
print(data_df.head())

In [ ]:
def dump_to_dataframe():
    try:
        # Define the SQL query
        query = """
        SELECT
            P.ProductID,
            P.ProductName,
            P.Description,
            P.Price,
            P.Category AS ProductCategory,
            P.StockQuantity,
            P.Manufacturer,
            P.ReleaseDate,
            P.ImageURL,
            C.CustomerID,
            C.FirstName,
            C.LastName,
            C.Email,
            C.Phone,
            C.Address,
            C.Username AS CustomerUsername,
            C.RegistrationDate,
            O.OrderID,
            O.OrderDate,
            O.TotalAmount AS OrderTotalAmount,
            O.OrderStatus,
            O.CustomerID AS OrderCustomerID,
            OI.OrderItemID,
            OI.Quantity AS OrderItemQuantity,
            OI.ItemPrice AS OrderItemPrice,
            OI.ProductID AS OrderItemProductID,
            Pmt.PaymentID,
            Pmt.PaymentDate,
            Pmt.PaymentAmount AS PaymentAmount,
            Pmt.PaymentMethod,
            Pmt.TransactionID,
            R.ReviewID,
            R.Rating AS ReviewRating,
            R.Comment AS ReviewComment,
            R.ReviewDate,
            Cat.CategoryID AS ProductCategoryID,
            Cat.CategoryName AS ProductCategoryName,
            Cat.ParentCategoryID
        FROM Products P
        JOIN OrderItems OI ON P.ProductID = OI.ProductID
        JOIN Orders O ON OI.OrderID = O.OrderID
        JOIN Customers C ON O.CustomerID = C.CustomerID
        LEFT JOIN Payments Pmt ON O.OrderID = Pmt.OrderID
        LEFT JOIN Reviews R ON P.ProductID = R.ProductID
        LEFT JOIN Categories Cat ON P.Category = Cat.CategoryName;
        """
        
        # Execute the query and fetch the result
        cursor.execute(query)
        result = cursor.fetchall()
        
        # Get column names from the cursor description
        columns = [desc[0] for desc in cursor.description]
        
        # Create a Pandas DataFrame from the result
        df = pd.DataFrame(result, columns=columns)
        
        return df

    except Exception as e:
        print("Error executing SQL query:", e)

# Example usage
dataframe = dump_to_dataframe()

# work with the data in the Pandas DataFrame
# use DataFrame functions like .head(), .describe(), .info(), etc.
#print(dataframe.head(1))
print(dataframe.columns)
#print(dataframe.info())

In [ ]:
# Close the database connection
cursor.close()
db_conn.close()

In [ ]:
# example logic for shell-based shop.
while True:
    print("\nWelcome to Simple Shop")
    print("\nWhat do you want to do. \nOptions:")
    print("1. Create User")
    print("2. Get Product List")
    print("3. Get Product Details")
    print("4. Get Product Reviews")
    print("5. Create Order")
    print("6. Make Payment and Change Order Status")
    print("7. Exit")

    option = input("Select an option: ")

    if option == "1":
        # Option to create a user
        username = input("Username: ")
        password = input("Password: ")
        first_name = input("First Name: ")
        last_name = input("Last Name: ")
        email = input("Email: ")
        phone = input("Phone: ")
        address = input("Address: ")
        # Call the create_consumer function here

    elif option == "2":
        # Option to get a product list
        # Call the list_all_products function here
        pass

    elif option == "3":
        # Option to get product details by product ID
        product_id = input("Enter Product ID: ")
        # Call the read_product function with the product_id

    elif option == "4":
        # Option to get product reviews by product ID
        product_id = input("Enter Product ID: ")
        # Call the display_product_reviews function with the product_id

    elif option == "5":
        # Option to create an order
        customer_id = input("Enter Customer ID: ")
        product_id = input("Enter Product ID: ")
        quantity = input("Enter Quantity: ")
        # Call the create_order function here

    elif option == "6":
        # Option to make a payment and change order status
        order_id = input("Enter Order ID: ")
        payment_amount = input("Enter Payment Amount: ")
        payment_method = input("Enter Payment Method: ")
        transaction_id = input("Enter Transaction ID: ")
        # Call the make_payment function here
        # Call the change_order_status function here

    elif option == "7":
        # Option to exit the program
        break

    else:
        print("Invalid Option. Press 1-7")


In [ ]:
# Function to perform basic input sanitation (use default values if not provided)
# need to be extended for datatype check, REGEX check etc.
def sanitize_input(input_value, default_value):
    return input_value if input_value else default_value

if True:        
    product_data = {}
    product_data["ProductName"] = input("Product Name (eg.'Amplifier') : ")
    product_data["Description"] = input("Product Description (eg.'Precision Power Amplifier') : ")
    product_data["Price"] = float(input("Product Price (eg.'99.99') : "))
    product_data["Category"] = input("Product Category (eg.'Electronics') : ")
    product_data["StockQuantity"] = int(input("Stock Quantity (eg.'10') : "))
    product_data["Manufacturer"] = input("Manufacturer Name (eg.'Diora') : ")
    product_data["ReleaseDate"] = input("Release Date (YYYY-MM-DD): ")
    product_data["ImageURL"] = input("Image URL (eg.'sample_image.jpg') : ")

    # Perform input sanitation (use default values if not provided)
    product_data["ProductName"] = sanitize_input(product_data["ProductName"], "noName")
    product_data["Description"] = sanitize_input(product_data["Description"], "No Description")
    product_data["Price"] = sanitize_input(product_data["Price"], 0.0)
    product_data["Category"] = sanitize_input(product_data["Category"], "")
    product_data["StockQuantity"] = sanitize_input(product_data["StockQuantity"], 0)
    product_data["Manufacturer"] = sanitize_input(product_data["Manufacturer"], "noManufacturer")
    # Set the ReleaseDate to the current date and time= 
    product_data["ReleaseDate"] = sanitize_input(product_data["ReleaseDate"], datetime.now().strftime("%Y-%m-%d"))
    product_data["ImageURL"] = sanitize_input(product_data["ImageURL"], "no_image.jpg")

    product_id = add_product(
            (product_data["ProductName"], product_data["Description"], product_data["Price"],
             product_data["Category"], product_data["StockQuantity"], product_data["Manufacturer"],
             product_data["ReleaseDate"], product_data["ImageURL"])
    )

In [ ]:
import argparse

# Initialize the argument parser
parser = argparse.ArgumentParser(description="Online Merchandise Store CLI")

# Subparsers for different options
subparsers = parser.add_subparsers(dest="command")

# Option to create a user
create_user_parser = subparsers.add_parser("create_user")
create_user_parser.add_argument("--username", required=True)
create_user_parser.add_argument("--password", required=True)
create_user_parser.add_argument("--first_name", required=True)
create_user_parser.add_argument("--last_name", required=True)
create_user_parser.add_argument("--email", required=True)
create_user_parser.add_argument("--phone")
create_user_parser.add_argument("--address")

# Option to get a product list
list_products_parser = subparsers.add_parser("list_products")

# Option to get product details by product ID
get_product_details_parser = subparsers.add_parser("get_product_details")
get_product_details_parser.add_argument("product_id", type=int)

# Option to get product reviews by product ID
get_product_reviews_parser = subparsers.add_parser("get_product_reviews")
get_product_reviews_parser.add_argument("product_id", type=int)

# Option to create an order
create_order_parser = subparsers.add_parser("create_order")
create_order_parser.add_argument("--customer_id", required=True, type=int)
create_order_parser.add_argument("--product_id", required=True, type=int)
create_order_parser.add_argument("--quantity", required=True, type=int)

# Option to make a payment and change order status
make_payment_parser = subparsers.add_parser("make_payment")
make_payment_parser.add_argument("--order_id", required=True, type=int)
make_payment_parser.add_argument("--payment_amount", required=True, type=float)
make_payment_parser.add_argument("--payment_method", required=True)
make_payment_parser.add_argument("--transaction_id", required=True)

# Parse the command-line arguments
args = parser.parse_args()

# Connect to the MySQL server
db_conn = mysql.connector.connect(**db_config)
cursor = db_conn.cursor()

if args.command == "create_user":
    # Handle the "create_user" option
    password_hash = bcrypt.hashpw(args.password.encode("utf-8"), bcrypt.gensalt())
    create_consumer(args.username, password_hash, args.first_name, args.last_name, args.email, args.phone, args.address)

elif args.command == "list_products":
    # Handle the "list_products" option
    products = list_all_products()
    for product in products:
        print(product)

elif args.command == "get_product_details":
    # Handle the "get_product_details" option
    product_info = read_product(args.product_id)
    if product_info:
        print("Product Information:", product_info)
    else:
        print("Product not found.")

elif args.command == "get_product_reviews":
    # Handle the "get_product_reviews" option
    display_product_reviews(args.product_id)

elif args.command == "create_order":
    # Handle the "create_order" option
    create_order(args.customer_id, args.product_id, args.quantity)

elif args.command == "make_payment":
    # Handle the "make_payment" option
    make_payment(args.order_id, args.payment_amount, args.payment_method, args.transaction_id)

    # Change order status to "Shipped" (or any desired status)
    change_order_status(args.order_id, "Shipped")

In [ ]:
#  run the CLI by providing the appropriate options and arguments
# python merchandise_store_cli.py create_user --username john_doe --password secret --first_name John --last_name Doe --email john@example.com
# python merchandise_store_cli.py list_products
# python merchandise_store_cli.py get_product_details 1
# python merchandise_store_cli.py get_product_reviews 1
# python merchandise_store_cli.py create_order --customer_id 1 --product_id 2 --quantity 3
# python merchandise_store_cli.py make_payment --order_id 1 --payment_amount 150.00 --payment_method "Credit Card" --transaction_id "1234567890"